In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neural_network import MLPClassifier

In [0]:
def createXorData(samples=100000, size=50):
  """
  generates a dataset for the XOR problem in 
    https://blog.openai.com/requests-for-research-2/
  LHS are sequences of bools
  RHS computes the parity bit 
    (eg. wether the sequence has odd or even sum)
  """
  lhs = np.zeros((samples, size), dtype=np.int8)
  rhs = np.zeros(samples)
  for row in range(samples):
    lhs[row] = np.random.randint(0, 2, size)
    rhs[row] = lhs[row].sum() % 2
  return lhs, rhs

In [0]:
SIZE = 50
SAMPLES = 20000
LEARNING_RATE = 0.1
TRAIN_STEPS = 2500
BATCH_SIZE = 1000

Xtrain, Ytrain = createXorData(SAMPLES, SIZE)
Xtest, Ytest = createXorData(int(SAMPLES/2), SIZE)

In [37]:
#
# ridiculous MLP
#

# TODO: try mlp with cosinus non-linearity
mlp = MLPClassifier(hidden_layer_sizes=(50, 25, 10), 
                    activation="relu"
                   ).fit(Xtrain, Ytrain)
pred_score = mlp.score(Xtest, Ytest)
print(pred_score)

0.4956


In [0]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [47]:
#
# single layer NN in TF
#
sess = tf.InteractiveSession()
# device should be in /cpu:0 or /gpu:0
with tf.device('/gpu:0'):
  x_tf = tf.placeholder(tf.float32, shape=[None, SIZE])
  y_tf = tf.placeholder(tf.float32, shape=[None])

  Weights = tf.Variable(tf.zeros([SIZE,1]))
  Biases = tf.Variable(tf.zeros([1]))

  yhat = tf.cos(tf.matmul(x_tf, Weights) + Biases)

  cross_entropy_loss  = tf.reduce_mean(-tf.reduce_sum(y_tf * tf.log(yhat),
                                       reduction_indices=[1]))

  train_step = tf.train.AdamOptimizer(
                          LEARNING_RATE).minimize(cross_entropy_loss)

init = tf.global_variables_initializer()
sess.run(init)

for _ in range(10000):
  batch_xs, batch_ys = next_batch(1000, Xtrain, Ytrain)
  sess.run(train_step, feed_dict={x_tf: batch_xs, y_tf: batch_ys})
  
correct_prediction = tf.equal(
    tf.cast(tf.greater_equal(yhat, 0.5), tf.float32),
    y_tf)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={x_tf: Xtest, y_tf: Ytest}))

Accuracy:  0.50719994


In [50]:
#
# FFNN in TF
#
sess = tf.InteractiveSession()
# device should be in /cpu:0 or /gpu:0
with tf.device('/gpu:0'):
  x_tf = tf.placeholder(tf.float32, shape=[None, SIZE])
  y_tf = tf.placeholder(tf.float32, shape=[None])
 
  
  weights = {
      'h1': tf.Variable(tf.zeros([SIZE,SIZE])),
      'h2': tf.Variable(tf.zeros([SIZE,SIZE])),
      'out': tf.Variable(tf.zeros([SIZE,1])),
  }
  biases = {
      'b1':tf.Variable(tf.zeros([SIZE])),
      'b2':tf.Variable(tf.zeros([SIZE])),
      'out':tf.Variable(tf.zeros([1])),
  }
  
  def MLP(x):
    # Hidden fully connected layers
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output Cos of X'W +b
    out_layer = tf.cos(tf.matmul(layer_2, weights['out']) + biases['out'])
    return out_layer
  
  tf_mlp = MLP(x_tf)
  
  loss_op = tf.reduce_mean(-tf.reduce_sum(y_tf * tf.log(tf_mlp), reduction_indices=[1]))
  
  init = tf.global_variables_initializer()
  sess.run(init)

  for _ in range(10000):
    batch_xs, batch_ys = next_batch(1000, Xtrain, Ytrain)
    sess.run(train_step, feed_dict={x_tf: batch_xs, y_tf: batch_ys})

  correct_prediction = tf.equal(
      tf.cast(tf.greater_equal(yhat, 0.5), tf.float32),
      y_tf)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print("Accuracy: ", sess.run(accuracy, feed_dict={x_tf: Xtest, y_tf: Ytest}))

InvalidArgumentError: ignored